# The important parts of the code

## Command imports

In [129]:
%pylab inline 
import os
import nibabel as nib
import numpy as np
import glob
from scipy.ndimage import label
from numpy.linalg import inv

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Import mse IDs 

In [130]:
#subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
subjects = ["mse1406"]
all_sub_results = [["mseID",
                   "lesion number and type",      
                   "number of subcortical lesions",
                   "number of juxtacortical lesions",
                   "number of periventricular lesions",
                   "number of infratentorial lesions",
                   "number of errors"]]
print subjects[:5]

['mse1406']


## Set variables to lesion/segmentation files

In [131]:
for les_file in glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/alignment_lesions.nii.gz" % subjects[0]):
    print les_file
for seg_file in glob.glob("/data/henry7/PBR/subjects/%s/masks/*/segmentation.nii.gz" % subjects[0]):
    print seg_file
    
print seg_file, les_file

/data/henry7/PBR/subjects/mse1406/lesions_manual/ms172-mse1406-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz
/data/henry7/PBR/subjects/mse1406/masks/ms172-mse1406-002-AX_T1_3D_IRSPGR/segmentation.nii.gz
/data/henry7/PBR/subjects/mse1406/masks/ms172-mse1406-002-AX_T1_3D_IRSPGR/segmentation.nii.gz /data/henry7/PBR/subjects/mse1406/lesions_manual/ms172-mse1406-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz


## Obtain affines 

In [132]:
les_img = nib.load(les_file)
les_img.dataobj
seg_img = nib.load(seg_file)
seg_img.dataobj

les_data = les_img.get_data()
print les_data.shape
seg_data = seg_img.get_data()
print seg_data.shape

les_affine = les_img.get_affine()
print les_affine
seg_affine = seg_img.get_affine()
print seg_affine
inv_seg_affine = np.linalg.inv(seg_affine)
print inv_seg_affine

(256, 256, 149)
(256, 256, 256)
[[  -0.9375        0.            0.          121.8730011 ]
 [   0.            0.9375        0.          -86.33650208]
 [   0.            0.            1.          -60.81729889]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          129.8730011 ]
 [   0.            0.            1.          -94.33650208]
 [   0.           -1.            0.          157.18270874]
 [   0.            0.            0.            1.        ]]
[[  -1.           -0.           -0.          129.8730011 ]
 [  -0.           -0.           -1.          157.18270874]
 [   0.            1.            0.           94.33650208]
 [   0.            0.            0.            1.        ]]


## set labels for coordinates

In [133]:
#lesion labels
les_labels, n_les_labels = label(les_data==[1])

#segmentation labels
seg_brainstem_labels, n_seg_brainstem_labels = label(seg_data==[16])
seg_lcerebellumcortex_labels, n_seg_lcerebellumcortex_labels = label(seg_data==[8])
seg_rcerebellumcortex_labels, n_seg_rcerebellumcortex_labels = label(seg_data==[47])
seg_lcerebellumwm_labels, n_seg_lcerebellumwm_labels = label(seg_data==[7])
seg_rcerebellumwm_labels, n_seg_rcerebellumwm_labels = label(seg_data==[46])

## debugging tools for labels

In [134]:
print n_les_labels #prints number of lesion labels

print n_seg_brainstem_labels #prints number of labels in brainstem

print np.bincount(les_labels.ravel()) #prints volumes of lesions

print np.bincount(seg_lcerebellumwm_labels.ravel()) #prints volume of left cerebellum white matter

17
1
[9762255     521     172     149      53      56      54     125      24
      65      45     436      25      36     256      93      28     471]
[16764422    12794]


## Function that gets coordinates from labels

In [135]:
def get_label_coord(labels,num_labels):
    all_label_coords = []
    if num_labels >= 1:
        for count in range(1, num_labels+1):
            cur_label_coords = []
            x,y,z = np.nonzero(labels==count)
            for count2 in range(len(x)):
                cur_coord = [x[count2],y[count2],z[count2]]
                cur_label_coords.append(cur_coord)
            all_label_coords.append(cur_label_coords)
    else:
        x,y,z = np.nonzero(labels==1)
        for count in range(len(x)):
            all_label_coords.append([x[count],y[count],z[count]])
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_les_coords[count]
    #    print ""
    return all_label_coords

## Function that converts coordinates to real space

In [136]:
def get_rs_coord(coordinates):
    all_rs_coords = []
    for count in range(len(coordinates)):
        cur_rs_coords = []
        for count2 in range(len(coordinates[count])):
            rs_coord = np.dot(les_affine, [coordinates[count][count2][0],
                                           coordinates[count][count2][1], 
                                           coordinates[count][count2][2],
                                           1])
            rs_coord_noone = [rs_coord[0],rs_coord[1],rs_coord[2]] #raw float
            #rs_coord_noone = [round(rs_coord[0],2),round(rs_coord[1],2),round(rs_coord[2],2)] #two decimal float
            #rs_coord_noone = [int(rs_coord[0]),int(rs_coord[1]),int(rs_coord[2])] #integers
            cur_rs_coords.append(rs_coord_noone)
        all_rs_coords.append(cur_rs_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_rs_coords[count]
    #    print ""
    return all_rs_coords

## Function that converts coordinates to segmentation file coordinates

In [137]:
def get_seg_coord(coordinates):
    all_seg_coords = []
    for count in range(len(coordinates)):
        cur_seg_coords = []
        for count2 in range(len(coordinates[count])):
            seg_coord = np.dot(inv_seg_affine, [coordinates[count][count2][0],
                                                coordinates[count][count2][1], 
                                                coordinates[count][count2][2],
                                                1])
            #seg_coord_noone = [seg_coord[0],seg_coord[1],seg_coord[2]] #raw float
            #seg_coord_noone = [round(seg_coord[0],2),round(seg_coord[1],2),round(seg_coord[2],2)] #two decimal float
            seg_coord_noone = [int(seg_coord[0]),int(seg_coord[1]),int(seg_coord[2])] #integers
            cur_seg_coords.append(seg_coord_noone)
        all_seg_coords.append(cur_seg_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_seg_coords[count]
    #    print ""
    return all_seg_coords

## Generate midbrain coordinates into a variable

In [138]:
midbrain_int = []
bs = get_label_coord(seg_brainstem_labels,n_seg_brainstem_labels)
lcc = get_label_coord(seg_lcerebellumcortex_labels,n_seg_lcerebellumcortex_labels)
lcw = get_label_coord(seg_lcerebellumwm_labels,n_seg_lcerebellumwm_labels)
rcc = get_label_coord(seg_rcerebellumcortex_labels,n_seg_rcerebellumcortex_labels)
rcw = get_label_coord(seg_rcerebellumwm_labels,n_seg_rcerebellumwm_labels)

midbrain = []

if len(bs) != 1:
    midbrain.append(bs[1])
else:
    midbrain.append(bs[0])

if len(lcc) != 1:
    midbrain.append(lcc[1])
else:
    midbrain.append(lcc[0])

if len(lcw) != 1:
    midbrain.append(lcw[1])
else:
    midbrain.append(lcw[0])

if len(rcc) != 1:
    midbrain.append(rcc[1])
else:
    midbrain.append(rcc[0])

if len(rcw) != 1:
    midbrain.append(rcw[1])
else:
    midbrain.append(rcw[0])

print len(midbrain)



5


## Generate lesion coordinates into a variable

In [139]:
lesions = get_seg_coord(get_rs_coord(get_label_coord(les_labels,n_les_labels)))
lesions_les = get_label_coord(les_labels,n_les_labels)
lesions_seg = lesions
"""
print len(lesions)
print lesions_seg[0][1]
print lesions_les[7]
print lesions_seg[7]
print lesions_les[7][39]
print lesions_seg[7][39]
"""

'\nprint len(lesions)\nprint lesions_seg[0][1]\nprint lesions_les[7]\nprint lesions_seg[7]\nprint lesions_les[7][39]\nprint lesions_seg[7][39]\n'

## Infratentorial Lesion Check

les_type = []
for count in range(len(lesions)):
    lesion_type = ""
    count4 = 0
    if len(lesions[count]) <= 500:
        for count2 in range(len(lesions[count])):
            halt = ""
            for count3 in range(len(midbrain)):
                for count4 in range(len(midbrain[count3])):
                    if lesions[count][count2] == midbrain[count3][count4]:
                        lesion_type = "infratentorial"
                        halt = "stop"
                    if halt == "stop":
                        continue
                if halt == "stop":
                    continue
            if halt == "stop":
                continue
        if lesion_type == "":
            lesion_type = "subcortical"
        print count, "done, type is:", lesion_type
        les_type.append(lesion_type)
    else:
        print "skipped lesion number", count, "; too large"
        les_type.append("too large")

print les_type

## Convert coordinates from nested lists to strings

In [140]:
def convert_coord_str(coordinates):
    coord_str = []
    for x in range(len(coordinates)):
        str_int = []
        for y in range(len(coordinates[x])):
            str_int.append(str(coordinates[x][y]))
        coord_str.append(str_int)
    return coord_str

lesions_str = convert_coord_str(lesions)
midbrain_str = convert_coord_str(midbrain)

## Infratentorial Check V2 

In [143]:
les_type = []
count = 0
while count <= (len(lesions_str))-1:
    lesion_type = ""
    x = set(lesions_str[count])
    count2 = 0
    while count2 <= (len(midbrain_str))-1:
        y = set(midbrain_str[count2])
        testint = x.isdisjoint(y)
        if testint == False:
            count2 = len(midbrain_str)
        count2 += 1
    if testint == False:
        lesion_type = "infratentorial"
    else:
        lesion_type = "subcortical"
    print count+1, lesion_type
    les_type.append(lesion_type)
    count += 1

1 subcortical
2 subcortical
3 subcortical
4 subcortical
5 subcortical
6 subcortical
7 subcortical
8 subcortical
9 infratentorial
10 subcortical
11 subcortical
12 subcortical
13 subcortical
14 subcortical
15 subcortical
16 subcortical
17 subcortical


## append results to a new list 

In [144]:
results = []
sub_count, inf_count, jux_count, per_count, err_count = 0, 0, 0, 0, 0
for count in range(len(les_type)):
    results.append([count+1, les_type[count],])
    if les_type[count] == "subcortical":
        sub_count += 1
    elif les_type[count] == "infratentorial":
        inf_count += 1
    elif les_type[count] == "juxtacortical":
        jux_count += 1
    elif les_type[count] == "periventricular":
        per_count += 1
    else:
        err_count += 1
all_sub_results.append([subjects[0], 
                        results, 
                        sub_count, 
                        jux_count,
                        per_count,
                        inf_count, 
                        err_count]
                      )
print all_sub_results

[['mseID', 'lesion number and type', 'number of subcortical lesions', 'number of juxtacortical lesions', 'number of periventricular lesions', 'number of infratentorial lesions', 'number of errors'], ['mse1406', [[1, 'subcortical'], [2, 'subcortical'], [3, 'subcortical'], [4, 'subcortical'], [5, 'subcortical'], [6, 'subcortical'], [7, 'subcortical'], [8, 'subcortical'], [9, 'infratentorial'], [10, 'subcortical'], [11, 'subcortical'], [12, 'subcortical'], [13, 'subcortical'], [14, 'subcortical'], [15, 'subcortical'], [16, 'subcortical'], [17, 'subcortical']], 16, 0, 0, 1, 0]]


# The debugging portions of the code

## Debugging function that finds range of values in brain lesion/segment

In [145]:
test_x = [];test_y=[];test_z=[]
for count in range(len(midbrain[0])):
    test_x.append(midbrain[0][count][0])
    test_y.append(midbrain[0][count][1])
    test_z.append(midbrain[0][count][2])

def range_coord(val_list):
    min_val = min(val_list)
    max_val = max(val_list)
    print min_val,"-", max_val

range_coord(test_x);range_coord(test_y);range_coord(test_z)

113 - 147
153 - 205
91 - 120


## Debugging Function that searches for coordinates in any given brain lesion/segment

In [146]:
def search(coordinates,check_coord):
    number = 0
    for count in range(len(coordinates)):
        if check_coord == coordinates[count]:
            number = count + 1
            print count
    if number == 0:
        print "not available"
    else:
        print "available"

#search(midbrain[0],lesions[7][39])
#search(midbrain[0],[144, 177, 100])
#search(lesions[7],lesions[7][39])

## Attempt at understanding "any" function

In [147]:
test = any(count for lesions[7][39] in midbrain)
print test

<generator object <genexpr> at 0x7f591a47ec30>


## Function that takes in coordinates for lesions, and outputs average; may not actually use this

In [148]:
def average_func(coordinates):
    averages_all = []
    for count in range(len(coordinates)):
        averages = []
        sumx=0;sumy=0;sumz=0
        for count2 in range(len(coordinates[count])):
            sumx += coordinates[count][count2][0]
            sumy += coordinates[count][count2][1]
            sumz += coordinates[count][count2][2]
        average_x = sumx / len(coordinates[count])
        average_y = sumy / len(coordinates[count])
        average_z = sumz / len(coordinates[count])
        averages = [average_x, average_y, average_z]
        averages_all.append(averages)
        #print count, averages
    return averages_all

In [149]:
average_les = average_func(get_label_coord(les_labels, n_les_labels))
average_rs = average_func(get_rs_coord(get_label_coord(les_labels, n_les_labels)))
average_seg = average_func(get_seg_coord(get_rs_coord(get_label_coord(les_labels, n_les_labels))))

allcoord_les = get_label_coord(les_labels, n_les_labels)
allcoord_rs = get_rs_coord(get_label_coord(les_labels, n_les_labels))
allcoord_seg = get_seg_coord(get_rs_coord(get_label_coord(les_labels, n_les_labels)))

print average_les[0]
print average_rs[0]
print average_seg[0]
print allcoord_les[0][0]
print allcoord_rs[0][0]
print allcoord_seg[0][0]

print average_seg

print lesions[8][20]

[86, 94, 93]
[40.396873459477341, 2.3319240284515206, 32.794985179937733]
[88, 124, 96]
[79, 94, 96]
[47.810501098632812, 1.7884979248046875, 35.182701110839844]
[82, 122, 96]
[[88, 124, 96], [99, 134, 79], [100, 133, 147], [101, 143, 62], [102, 151, 64], [108, 141, 158], [110, 156, 148], [115, 105, 141], [126, 193, 104], [136, 121, 108], [145, 141, 156], [144, 96, 132], [146, 131, 68], [150, 123, 90], [152, 138, 60], [155, 150, 153], [159, 154, 69]]
[126, 193, 102]


## Debugging Coding Section

In [150]:
realspace_coordinates = np.dot(les_affine, [average_les[0][0],
                 average_les[0][1], 
                 average_les[0][2],
                 1])
average_les[0] 
realspace_coordinates
from numpy.linalg import inv
inv_seg_affine = np.linalg.inv(seg_affine)
freesurfer_matrix = np.dot(inv_seg_affine, [realspace_coordinates[0],
                                            realspace_coordinates[1], 
                                            realspace_coordinates[2],
                                            1])
print average_les[0]; print realspace_coordinates; print freesurfer_matrix

print len(midbrain[0])

les_type = []

"""for test_count in range(len(lesions)):
    for count in range(len(lesions[test_count])):
        for count2 in range(len(midbrain)):
            halt = ""
            for count3 in range(len(midbrain[count2])):
                if lesions[test_count][count] == midbrain[count2][count3]:
                    inf_num += 1
                    lesion_type = "infratentorial"
                    halt = "stop"
                    break
            if halt == "stop":
                break
    print lesion_type
if lesion_type == "infratentorial":
    les_type.append(lesion_type)
elif lesion_type == "":
    les_type.append("subcortical")
print les_type
print inf_num
"""

[86, 94, 93]
[ 41.2480011    1.78849792  32.18270111   1.        ]
[  88.625       125.00000763   96.125         1.        ]
16685


'for test_count in range(len(lesions)):\n    for count in range(len(lesions[test_count])):\n        for count2 in range(len(midbrain)):\n            halt = ""\n            for count3 in range(len(midbrain[count2])):\n                if lesions[test_count][count] == midbrain[count2][count3]:\n                    inf_num += 1\n                    lesion_type = "infratentorial"\n                    halt = "stop"\n                    break\n            if halt == "stop":\n                break\n    print lesion_type\nif lesion_type == "infratentorial":\n    les_type.append(lesion_type)\nelif lesion_type == "":\n    les_type.append("subcortical")\nprint les_type\nprint inf_num\n'